In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import os
import pickle

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [13]:
df_file  = 'SW_Principales_BTC.pkl'

with open(df_file, 'rb') as file:
    df = pickle.load(file)
    #data.set_index('datetime',inplace=True)
    df.drop(columns=['open','high','low','close','volume',],inplace=True)

In [ ]:
#Ajustando valores del dataframe

#ffill
df["stock_total"] = df["stock_total"].fillna(method="ffill")
df["stock_quote"] = df["stock_quote"].fillna(method="ffill")
df["valor_capital"] = df["valor_capital"].fillna(method="ffill")
df["precio_promedio"] = df["precio_promedio"].fillna(method="ffill")
df["ganancias_realizadas"] = df["ganancias_realizadas"].fillna(method="ffill")

#Calculos
df["valor_stock"] = df["stock_total"]*df['price'] 
df['valor_actual_total'] = df['stock_quote'] + df['valor_stock']
df["rendimiento_ganancias"] = (df["ganancias_realizadas"] / df['valor_capital'] )*100
df["rendimiento_valor"] = (df["valor_actual_total"] / df['valor_capital'] - 1 )*100

#Eventos
df['buy'] = np.where((df['side']==0)&(df['capital'] == False),df['price'],None)
df['sell'] = np.where((df['side']==1)&(df['capital'] == False),df['price'],None)
df['aporte'] = np.where((df['side']==0)&(df['capital'] == True),df['price'],None)
df['retiro'] = np.where((df['side']==1)&(df['capital'] == True),df['price'],None)


In [88]:
#Creando Chart 
chart_rows = 3
row_heights=[300,150,150]
total_height = sum(row_heights)

fig = make_subplots(rows=chart_rows, 
                    shared_xaxes=True,
                    row_heights=row_heights)

fig.add_trace(
        go.Scatter(
            x=df["datetime"], y=df["price"], name=f'Precio', mode="lines",  
            line={'width': 0.5},  
            marker=dict(color='gray'),
        ),
        row=1,
        col=1,
    )  
fig.add_trace(
        go.Scatter(
            x=df["datetime"], y=df["precio_promedio"], name=f'Precio Promedio', mode="lines",  
            line={'width': 0.5},  
            marker=dict(color='#f8b935'),
        ),
        row=1,
        col=1,
    )  
fig.add_trace(
        go.Scatter(x=df["datetime"], y=df['buy'], name='Compras', mode='markers', 
                marker=dict(symbol='circle',size=4,color='#0ecb81',line=dict(width=0.75, color="black"),),
                ),row=1,col=1,
    )
fig.add_trace(
        go.Scatter(x=df["datetime"], y=df['sell'], name='Ventas', mode='markers', 
                marker=dict(symbol='circle',size=4,color='#f6465d',line=dict(width=0.75, color="black"),),
                ),row=1,col=1,
    )
fig.add_trace(
        go.Scatter(x=df["datetime"], y=df['aporte'], name='Aportes', mode='markers', 
                marker=dict(symbol='triangle-up',size=8,color='#0ecb81',line=dict(width=0.75, color="black"),),
                ),row=1,col=1,
    )
fig.add_trace(
        go.Scatter(x=df["datetime"], y=df['retiro'], name='Retiros', mode='markers', 
                marker=dict(symbol='triangle-down',size=8,color='#f6465d',line=dict(width=0.75, color="black"),),
                ),row=1,col=1,
    )


fig.add_trace(
        go.Scatter(
            x=df["datetime"], y=df["ganancias_realizadas"], name=f'Ganancias', mode="lines",  
            line={'width': 0.5},  
            marker=dict(color='#0dcaf0'),
        ),
        row=2,
        col=1,
    )  

fig.add_trace(
        go.Scatter(
            x=df["datetime"], y=df["rendimiento_ganancias"], name=f'Ganancias (%)', mode="lines",  
            line={'width': 0.5},  
            marker=dict(color='#fcd535'),
        ),
        row=3,
        col=1,
    )  
fig.add_trace(
        go.Scatter(
            x=df["datetime"], y=df["rendimiento_valor"], name=f'Valor (%)', mode="lines",  
            line={'width': 0.5},  
            marker=dict(color='#fd7e14'),
        ),
        row=3,
        col=1,
    )  





# Adjust layout for subplots
fig.update_layout(
    font=dict(color="#ffffff", family="Helvetica"),
    paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
    plot_bgcolor="rgba(0,0,0,0)",  # Transparent plot area 
    height=total_height,
    xaxis_rangeslider_visible=False,
    modebar_bgcolor="rgba(0,0,0,0)",
    legend=dict(
        orientation = 'h',
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=0,
    ),
)

#Ajustar el tamaño de cada sub_plot
fig.update_layout(
    yaxis1=dict(title="Precio",showticklabels=True,
                zeroline=False,),
    yaxis2=dict(title="USDT",showticklabels=True,),
    yaxis3=dict(title="(%)",showticklabels=True,),
)
fig.update_xaxes(showline=True, linewidth=0.5,linecolor='#40444e', gridcolor='#40444e')
fig.update_yaxes(showline=False, linewidth=0.5, zeroline= False, linecolor='#40444e', gridcolor='rgba(40,40,40,10)') 

fig.show()